In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from scipy.io import loadmat
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy import stats
from scipy.stats import mannwhitneyu, ttest_rel
from scipy.signal import butter, filtfilt
import mne
import eeg_analysis.funcs4eeg as fe
import re
import imp
import ast
import behavior.func4behav as fb
imp.reload(fe)
imp.reload(fb)

from scipy import signal
from scipy.signal import resample
from scipy.ndimage import zoom

In [2]:
# combine fast and slow
case_title = 'endo'
case_list = [case_title+' fast', case_title+' slow']
tmax_list = [1.5+1+0.5+0.05+1, 1.5+1+1+0.05+1]

# case_title = 'exo'
# case_list = [case_title+' fast', case_title+' slow']
# tmax_list = [1.5+0.033*4+0.5+0.05+1, 1.5+0.033*4+1+0.05+1]

fs = 1200
eeg_fast = np.empty((0, 32, int(tmax_list[0]*fs)+1))
eeg_slow = np.empty((0, 32, int(tmax_list[1]*fs)+1))
rt = np.empty((0,))
for i, case in enumerate(case_list):
    tmax = tmax_list[i]
    watch = '1 fixation'
    tmin = 0 # include fix or not?

    highpass = 4 # 0.3
    lowpass = 50

    EP_lists, RT_lists = fe.pipeline_EP_RT(case, watch, tmin, tmax, hipass=highpass, lopass=lowpass, baseline=(0,0), move_baseline=False, detrend=1)
    eeg_tmp = np.vstack([EP_lists[0], EP_lists[2]]) # only for all before
    
    # make eeg to same length
    if 'slow' in case:
        eeg_slow = np.vstack((eeg_slow, eeg_tmp))
    else:
        eeg_fast = np.vstack((eeg_fast, eeg_tmp))

    rt_tmp = np.hstack([RT_lists[0], RT_lists[2]])
    rt = np.hstack((rt, rt_tmp))


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Filter length: 1981 samples (1.651 sec)



c:\ProgramData\anaconda3\envs\mne\lib\site-packages\mne\filter.py:312: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 317 samples (0.264 sec)



c:\ProgramData\anaconda3\envs\mne\lib\site-packages\mne\filter.py:312: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Filter length: 1981 samples (1.651 sec)



...

c:\ProgramData\anaconda3\envs\mne\lib\site-packages\mne\filter.py:312: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Filter length: 1981 samples (1.651 sec)



c:\ProgramData\anaconda3\envs\mne\lib\site-packages\mne\filter.py:312: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 317 samples (0.264 sec)



c:\ProgramData\anaconda3\envs\mne\lib\site-packages\mne\filter.py:312: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,


In [4]:
freq_bands = [[4, 8], [8, 12], [12, 30], [30, 50]]
endo_fast_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+0.5], [1.5+1+0.5, 1.5+1+0.5+0.05], [1.5+1+0.5+0.05, 1.5+1+0.5+0.05+1]]
endo_slow_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+1], [1.5+1+1, 1.5+1+1+0.05], [1.5+1+1+0.05, 1.5+1+1+0.05+1]]
exo_fast_stages = [[0, 1.5], [1.5, 1.5+0.033*4], [1.5+0.033*4, 1.5+0.033*4+0.5], [1.5+0.033*4+0.5, 1.5+0.033*4+0.5+0.05], [1.5+0.033*4+0.5+0.05, 1.5+0.033*4+0.5+0.05+1]]
exo_slow_stages = [[0, 1.5], [1.5, 1.5+0.033*4], [1.5+0.033*4, 1.5+0.033*4+1], [1.5+0.033*4+1, 1.5+0.033*4+1+0.05], [1.5+0.033*4+1+0.05, 1.5+0.033*4+1+0.05+1]]

bp_table = np.zeros((eeg_fast.shape[0]+eeg_slow.shape[0], 32, 5, 5)) # trials, 32 channels, 5 stages, 5 frequency bands

In [ ]:
def band_power(data, band):
    power = 0
    return power

eeg_list = [eeg_fast, eeg_slow]
if 'endo' in case_title:
    stages_list = [endo_fast_stages, endo_slow_stages]
else:
    stages_list = [exo_fast_stages, exo_slow_stages]

for i, eeg in enumerate(eeg_list):
    stages = stages_list[i]
    for trial in range(eeg.shape[0]):
        for channel in range(eeg.shape[1]):
            data = eeg[trial, channel, :]
            for stage in range(len(stages)):
                stage_start = int(stages[stage][0]*fs)
                stage_end = int(stages[stage][1]*fs)
                
                for band in range(len(freq_bands)):
                    
                    fmin = freq_bands[band][0]
                    fmax = freq_bands[band][1]
                    eeg_tmp = eeg[trial, ch, int(stages[stage][0]*fs):int(stages[stage][1]*fs)]
                    f, Pxx_den = signal.welch(eeg_tmp, fs, nperseg=fs, noverlap=0.5*fs, nfft=fs)
                    Pxx_den = Pxx_den[(f>=fmin) & (f<=fmax)]
                    compare_table[trial, ch, stage, band] = np.mean(Pxx_den)
